<center><h1>Cultural Analytics: Homework #3</h1></center>
<center><h2>ENGL 64.05 / QSS 30.16, 22F</h2></center>
<br>
<center><b>Due</b> 11:59PM 10/24/2022</center>

---

In [ ]:
# import required libraries

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation

from sklearn.feature_extraction import _stop_words as stop_words

from nltk.tokenize import sent_tokenize, word_tokenize

from glob import glob
import numpy as np

# <font color="red">Section 1: Single Text Topic Model</font>

In [ ]:
# Find, Download, and Upload a plain-text document to vectorize. 
# This should have at least several thousand tokens (i.e., a long text). Consider using Project Gutenberg or 
# another large text archive.

# make this variable's value the name of the file that you've uploaded
input_text = ""

# make into a collection of documents of 1,000 words
raw_text = open(input_text).read()
tokens = word_tokenize(raw_text)

segment_length = 1000
data_to_vectorize = list()

nseg = round(len(tokens) / segment_length)
for i in range(nseg):
    segment = tokens[segment_length*i:segment_length*(i+1)]
    data_to_vectorize.append(' '.join(segment))

# this will display some basic information about our data
print("created {0} segments of {1} words to vectorize from {2} total words".format(len(data_to_vectorize),
                                                                                   segment_length,
                                                                                   len(tokens)))

In [ ]:
# setup vectorizer and process text
vec = CountVectorizer(input='content',
                      stop_words='english',
                      lowercase=True)

dtm = vec.fit_transform(data_to_vectorize)
dc, vc = dtm.shape
print("read {0} documents with {1} vocabulary".format(dc,vc))

In [ ]:
# Build the LDA Model
#
# IMPORTANT PARAMETERS:
# n_components = number of topics to extract (if topics are too similar, extract more)
#

simple_lda_model = LatentDirichletAllocation(n_components=5,
                                             max_iter=5,
                                             learning_method='batch',
                                             random_state=1)    
# get fitted data and transformed matrix
simple_lda_data = simple_lda_model.fit(dtm)

# extract the features to a simple list
feature_names = vec.get_feature_names_out()

# IMPORTANT VARIABLES:
# how many words do we want to extract for each topic?
n_words = 100

In [ ]:
# now produce the list of topics
for topic_idx, topic in enumerate(simple_lda_model.components_):
    print("Topic #%d:" % topic_idx)
    for i in topic.argsort()[:-n_words - 1:-1]:
        print("{0} ".format(feature_names[i]),end="")
    print("\n")

<b>Question #1:</b>
In no more than two hundred words, what do you notice in the above list of words? Do you think these function as "topics?" Can you give each of them a name? Did you alter any of the variables? If so, what did you notice?

# <font color="red">Section 2: LDA on a Collection</font>


In [ ]:
# load some data from one of our collections (in the "data" directory)
# create a variable named "collection" that is a list of file names
# You'll need to use the glob() function to match some set of files. 
# Keep the number to no more than fifty files.
#
# You can use either Novel450 or the DocSouth collection.
# These are all located in the shared/ENGL64.05-22F/data directory

print("found",len(collection),"texts")

In [ ]:
segment_length = 1000

data_to_vectorize = list()
nc = 0
for document in collection:
    raw_text = open(document).read()
    tokens = word_tokenize(raw_text)

    nseg = round(len(tokens) / segment_length)
    for i in range(nseg):
        segment = tokens[segment_length*i:segment_length*(i+1)]
        data_to_vectorize.append(' '.join(segment))
    nc += 1
    
print("created {0} segments of {1} words to vectorize from {2} total texts".format(len(data_to_vectorize),
                                                                                   segment_length,
                                                                                   nc))

In [ ]:
# come back here later....
# custom stopword list...
# this loads the existing stopwords from Scikit-Learn
custom_stop_words = list(stop_words.ENGLISH_STOP_WORDS)

# now we need to add values to this list.
# how can you update an existing list?

In [ ]:
# stop_words can be 'english' or a variable name that is a list
vec = CountVectorizer(input='content',
                            stop_words='english',
                            lowercase=True) # lowercase is a Boolean True/False

dtm = vec.fit_transform(data_to_vectorize)
dc, vc = dtm.shape
print("read {0} documents with {1} vocabulary".format(dc,vc))

In [ ]:
# Build the LDA Model
#
# how many topics should we use?

lda_model = LatentDirichletAllocation(n_components=20,               
                                      max_iter=5,
                                      learning_method='batch',
                                      random_state=None)    
# get fitted data and transformed matrix
lda_data = lda_model.fit(dtm)

In [ ]:
# extract the features to a simple list
feature_names = vec.get_feature_names_out()

# how many words do we want to extract for each topic?
n_words = 100

# now produce topics
for topic_idx, topic in enumerate(lda_model.components_):
    print("Topic #%d:" % topic_idx)
    for i in topic.argsort()[:-n_words - 1:-1]:
        print("{0} ".format(feature_names[i]),end="")
    print("\n")

# Add Custom Stop Words

Now re-run the above by building a custom stoplist to remove some frequenly used words that you see in the above list.

In [ ]:
# Now we'll plot the words from the first topic
# You can easily modify this by selecting the component corresponding to the topic number of interest.

data = lda_model.components_[0]
word_index = data.argsort()[:-20 - 1:-1]

x = [feature_names[x] for x in word_index]
y = [data[x] for x in word_index]

import matplotlib.pyplot as plt
%matplotlib inline
plt.figure(figsize=(20,10))
plt.title("Topic #0: Important Tokens")

# what should these be?
# add labels...
plt.xlabel("")
plt.ylabel("")
plt.bar(x,y)
plt.show()

In [ ]:
# Create Document — Topic Matrix
import pandas as pd

lda_transformed_data = lda_model.fit_transform(dtm)

# make lists of topics and texts for labels
topics = ["Topic " + str(i) for i in range(lda_model.n_components)]
texts = ["Text " + str(i) for i in range(len(data_to_vectorize))]

# put data into a special datatype called a Pandas DataFrame
topic_chart = pd.DataFrame(np.round(lda_transformed_data, 3), columns=topics, index=texts)

# extract the most dominant topic by searching for largest value
topic_chart['Dominant Topic'] = np.argmax(topic_chart.values, axis=1)

# Display
topic_chart

In [ ]:
# We can select just those texts that are dominated by a topic of interest
topic_chart[topic_chart['Dominant Topic'] == 2]

<b>Question #2:</b>
In no more than two hundred words, respond the following questions: What do you notice about topic model on this collection of documents? What was the result of increasing the number of stop words? How many did you add and did that increase the interpretability of the model?

# <font color="red">Section 3: Non-Negative Matrix Factorization (NMF)</font>

We're now going to use a different method of generating a topic model. You can read more about this method below:

https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.NMF.html

We'll want to be looking for differences in the result of the model but make sure to note how we are using TF-IDF values instead of token counts.

In [ ]:
# convert to TF-IDF frequencies 
from sklearn.feature_extraction import text

tfidf = text.TfidfTransformer().fit_transform(dtm)

# fit to NMF model
from sklearn.decomposition import NMF

# n_components is the number of topics (same as LDA)
nmf_model = NMF(n_components=15,
                init='random', 
                random_state=0)

# fit 
nmf_output = nmf_model.fit_transform(tfidf)
nmf_data = nmf_model.fit(tfidf)

# reload feature names
feature_names = vec.get_feature_names_out()

# max number of words per topic to display
n_words = 100

for topic_idx, topic in enumerate(nmf_model.components_):
    print("Topic #%d:" % topic_idx)
    for i in topic.argsort()[:-n_words - 1:-1]:
        print("{0} ".format(feature_names[i]),end="")
    print("\n")

In [ ]:
# Create Document — Topic Matrix
import pandas as pd

# column names
topicnames = ["Topic " + str(i) for i in range(nmf_model.n_components)]

# index names
docnames = ["Text " + str(i) for i in range(len(data_to_vectorize))]

# Make the pandas dataframe
df_document_topic = pd.DataFrame(np.round(nmf_output, 2), columns=topicnames, index=docnames)

# Get dominant topic for each document
dominant_topic = np.argmax(df_document_topic.values, axis=1)
df_document_topic['Dominant Topic'] = dominant_topic
df_document_topic

<b>Question #3:</b>
In no more than two hundred words, respond the following questions: What differences do you see in the NMF model vs the LDA model? How might you determine which method to use in topic modeling?

In [ ]:
#
# This function will produce segments of a desired size from HathiTrust Extract Features data
#

from htrc_features import FeatureReader  

def get_segments(document, pos_nouns = False, segment_length=1000):
    fr = FeatureReader([document])
    vol = next(fr.volumes())
 
    # noun selector
    if pos_nouns == False:
        ptc = vol.tokenlist(pos=False, case=False).reset_index().drop(['section'], axis=1)
    else:
        ptc = vol.tokenlist(pos=True, case=False).reset_index().drop(['section'], axis=1)
        pos_tags = ["NN","NNS","NNP","NNPS"]
        ptc = ptc[ptc['pos'].isin(pos_tags)]
        ptc = ptc.drop('pos', axis=1)
        
    page_list = set(ptc['page'])
    
    # extract tokens by page 
    tokens=list()
    for page in page_list:
        page_data = str()
        
        # operate on each token
        for page_tokens in ptc.loc[ptc['page'] == page].iterrows():
            if page_tokens[1][1].isalpha():
                
                # deal with frequency count by creating correct number of tokens
                page_data += (' '.join([page_tokens[1][1]] * page_tokens[1][2])) + " "

        tokens = tokens + page_data.split()
    
    # now cut into segments
    data_to_vectorize = list()

    nseg = round(len(tokens) / segment_length)
    for i in range(nseg):
        segment = tokens[segment_length*i:segment_length*(i+1)]
        data_to_vectorize.append(' '.join(segment))    

    return data_to_vectorize

# <font color="red">Section 4: Topic Model of _The Dartmouth_ from HathiTrust</font>

You'll need to find the HathiTrust IDs for at least ten volumes of The Dartmouth at hathitrust.org. These IDs should contained as a list of strings in the texts_to_model variable.

<b>NOTE:</b> You will also need to add cells below to vectorize the tokens (data_to_vectorize) that we are extracting here and then create the topic models. You may use either the LDA or NMF model. You can select only nouns, if you wish by modifying the parameters for the get_segments function.

Once you've finished this, in no more than two hundred and fifty words, give an account of how successful you think this experiment in topic modeling has been. Are your models of these texts useful? How might you improve your results? What uses and limitations do you see in topic modeling?

In [ ]:
# Produce a list of HathiTrust IDs 

texts_to_model = 

# extract tokens and 
data_to_vectorize = list()
for text in texts_to_model:
    data_to_vectorize = data_to_vectorize + get_segments(text, segment_length=1000)